In [ ]:
import geopandas as gpd

geo_df = gpd.read_file('../geo_data/ref-nuts-2021/NUTS_RG_01M_2021_4326.geojson')
geo_df.rename(columns={'NUTS_ID': 'nuts_id'
                       , 'LEVL_CODE': 'levl_code'
                       , 'CNTR_CODE': 'cntr_code'
                       , 'NAME_LATN': 'name_latin'
                       , 'NUTS_NAME': 'nuts_name'
                       , 'MOUNT_TYPE': 'mount_type'
                       , 'URBN_TYPE': 'urban_type'
                       , 'COAST_TYPE': 'coast_type'
                       , 'FID': 'fid'
                       }
                , inplace=True)

geo_df = geo_df[geo_df['nuts_id'].str.contains("AT", case=False)]
geo_df

In [ ]:
from wifor_db import create_class

REGIONS = create_class('../wifor_db/tables/regions.json')
REGIONS.create_table()
REGIONS.add_data(geo_df, REGIONS.columns)

# Show
REGIONS.create_session().query(REGIONS).filter_by(nuts_id="AT").all()

In [ ]:
import eurostat
import pandas as pd

df = eurostat.get_data_df("lfst_r_lfe2en2", False)
df.rename(columns={'geo\\TIME_PERIOD': 'nuts_id'}, inplace=True)
df = df.melt(id_vars=['freq', 'nace_r2', 'age', 'sex', 'unit', 'nuts_id'], 
                  var_name='year', 
                  value_name='employment')
df['year'] = pd.to_datetime(df['year'], format='%Y')

df = df[df['nuts_id'].str.contains("AT", case=False)]
df

In [ ]:
from wifor_db import create_class

LFST_R_LFE2EN2 = create_class('../wifor_db/tables/lfst_r_lfe2en2.json')
LFST_R_LFE2EN2.create_table()
LFST_R_LFE2EN2.add_data(df, LFST_R_LFE2EN2.columns)

# Show
#LFST_R_LFE2EN2.create_session().query(LFST_R_LFE2EN2).all()

In [ ]:
LFST_R_LFE2EN2.create_session().query(LFST_R_LFE2EN2).filter_by(nuts_id="AT", nace_r2="A", age="Y15-24", sex='F', unit='THS_PER').all()

In [ ]:
from sqlalchemy import Column, String, ForeignKey
from sqlalchemy.orm import relationship

# Assuming LFST_R_LFE2EN2 and REGIONS are your existing model classes
# and they have been defined without relationships

# Add a relationship to LFST_R_LFE2EN2 to reference REGIONS
LFST_R_LFE2EN2.region = relationship("REGIONS", backref="lfst_r_lfe2en2s")

# Add a foreign key to LFST_R_LFE2EN2 if it doesn't exist
if 'nuts_id' not in LFST_R_LFE2EN2.__table__.c:
    nuts_id_column = Column(String(255), ForeignKey('REGIONS.nuts_id'))
    nuts_id_column.create(LFST_R_LFE2EN2.__table__, checkfirst=True)

# You can now use this relationship in your queries
session = LFST_R_LFE2EN2.create_session()
results = (session.query(LFST_R_LFE2EN2, REGIONS.nuts_name)
           .join(REGIONS, LFST_R_LFE2EN2.region)
           .filter(LFST_R_LFE2EN2.nuts_id == "AT")
           .all())

for record, nuts_name in results:
    print(f"Record: {record}, Region Name: {nuts_name}")


In [ ]:
results = (LFST_R_LFE2EN2.create_session()
           .query(LFST_R_LFE2EN2, REGIONS.nuts_name)
           .join(REGIONS, LFST_R_LFE2EN2.nuts_id == REGIONS.nuts_id)
           .filter_by(nuts_id="AT", nace_r2="A", age="Y15-24", sex='F', unit='THS_PER')
           .all())

results

In [ ]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()
db_user = os.environ.get('POSTGRES_DB_USER')
db_password = os.environ.get('POSTGRES_DB_PASSWORD')
db_host = os.environ.get('POSTGRES_DB_HOST')
db_name = os.environ.get('POSTGRES_DB_NAME')
db_port = os.environ.get('POSTGRES_DB_PORT', 5432)

if not all([db_user, db_password, db_host, db_name]):
    raise ValueError("PostgreSQL credentials are not set properly in .env file.")

db_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

create_engine(db_url, echo=False)

In [5]:
import os

for dirpath, dirnames, filenames in os.walk(os.getcwd()):
    if '.env' in filenames:
        dirpath

In [5]:
import os

def find_base_directory(current_dir, file_name="poetry.lock"):
    """
    Traverse up the directory tree to find the base directory based on the presence of a specific file.
    
    :param current_dir: The starting directory (typically where the script is).
    :param file_name: The name of the file to look for, default is 'poetry.lock'.
    :return: The absolute path to the base directory.
    """
    while not os.path.exists(os.path.join(current_dir, file_name)):
        parent_dir = os.path.dirname(current_dir)
        if parent_dir == current_dir:
            # We've reached the root directory without finding the file
            raise FileNotFoundError(f"Could not find the {file_name} file to identify the base directory.")
        current_dir = parent_dir
    return current_dir

# Usage
current_directory = os.getcwd()
base_directory = find_base_directory(current_directory)

base_directory

'c:\\Users\\HenryZeheWifOR\\OneDrive - WifOR\\WiFOR_Platform_Konzept\\WifOR_Platform'